### Preprocessing

In [1]:
import pickle

with open('embedding_labels.pkl', 'rb') as lf:
    labels = pickle.load(lf)

with open('embedding_posts.pkl', 'rb') as pf:
    posts = pickle.load(pf)
    
with open('posts_per_user.pkl', 'rb')as tpf:
    posts_per_user = pickle.load(tpf)

In [3]:
# Tokenizing data
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

max_features = 10000
max_len = 50
batch_size = 32
test_samples = 68965
max_words = 10000

Using TensorFlow backend.


In [4]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(posts)
sequences = tokenizer.texts_to_sequences(posts)
word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

Found 116582 unique tokens


In [12]:
data = sequence.pad_sequences(sequences, maxlen=max_len)
label_dictionary = {
    'I': 0,
    'E': 1
}

labels = [label_dictionary[label[:1]] for label in labels]

In [6]:
import numpy as np
labels = np.asarray(labels)

In [7]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (396704, 50)
Shape of label tensor: (396704,)


In [13]:
x_test = data[:test_samples]
y_test = labels[:test_samples]
x_train = data[test_samples:]
y_train = labels[test_samples:]

In [9]:
indices = np.arange(x_train.shape[0])
np.random.shuffle(indices)
x_train = x_train[indices]
y_train = y_train[indices]

In [14]:
from keras import models
from keras import layers
from keras.layers import Embedding, Flatten, LSTM, Dense

model = models.Sequential()
model.add(Embedding(max_features, max_len))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

# model.layers[0].set_weights([embedding_matrix])
# model.layers[0].trainable = False

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

model.fit(x_train, y_train, epochs=1, batch_size=8192)

Epoch 1/1
327739/327739 [==============================] - 72s 221us/step - loss: 0.5655 - acc: 0.7599


In [39]:
from pprint import pprint

def get_accuracy(posts_per_user, x_test, y_test):
    start_index = 0
    end_index = 28
    correct = []
    for i in range(0, 1500):
        predictions = model.predict(x_test[start_index:end_index])
        predictions = np.asarray(predictions).round()
        actual_label = int(y_test[start_index + 1])
        start_index = end_index
        end_index += posts_per_user[i+1]
        
        counter = 0
        for pred in predictions:
            counter += int(pred)
            
        pred_label = int(counter)
        
        if pred_label == actual_label:
            correct.append(1)
        else:
            correct.append(0)
        
    return correct

In [40]:
model_acc = get_accuracy(posts_per_user, x_test, y_test)

In [41]:
correct = 0
for pred in model_acc:
    correct += pred

correct / 1500

0.7733333333333333